In [2]:
# main.ipynb (Python cells for demonstration)

# Cell 1: Setup environment
import sys
import os
sys.path.append("src")

# Cell 2: Imports
from src.encoders import QuantumTimeSystem
from src.encoders import load_courses, load_instructors, load_groups, load_rooms
from src.ga_deap.runner import run_ga

# Cell 3: Initialize Time System and Load Data
qts = QuantumTimeSystem()
courses = load_courses("data/courses.json")
instructors = load_instructors("data/instructors.json", qts)
groups = load_groups("data/groups.json", qts)
rooms = load_rooms("data/rooms.json", qts)

# Cell 4: Print Loaded Overview
print("Courses loaded:", list(courses.keys()))
print("Instructors:", list(instructors.keys()))
print("Groups:", list(groups.keys()))
print("Rooms:", list(rooms.keys()))

# Cell 5: Run Genetic Algorithm
best_individual = run_ga(
    qts=qts,
    courses=courses,
    instructors=instructors,
    groups=groups,
    rooms=rooms,
    ngen=20,
    pop_size=50
)

# Cell 6: Inspect Result
print("Best fitness:", best_individual.fitness.values)
print("Sample gene (first 5):", best_individual[:5])


ValueError: Instructor I1: must be qualified for at least one course